# WiNDC National Model

You will need to point to the correct locations for a few objects, listed below

In [ ]:
parameter_path = raw"C:\Users\mphillipson\Documents\Modeling\MPSGE_dev_testing\nationaldata_julia\DAAData.jld2"
set_path = raw"C:\Users\mphillipson\Documents\Modeling\MPSGE_dev_testing\nationaldata_julia\Indices.jld2";

In [2]:
using MPSGE_MP

using CSV, DataFrames, JLD2

using JuMP

In [3]:
P= load(parameter_path)["data"] # load in date from saved Notebook output Dict, named P
S= load(set_path)["data"] # load in date from saved Notebook output Dict, named P


I = [i for i∈S[:i] if i∉[:use,:oth]]
J = [i for i∈S[:j] if i∉[:use,:oth]]
VA = [va for va∈S[:va] if va!=:othtax]
FD = S[:fd]
TS = S[:ts]
YR = S[:yr]
M = S[:m]

a_0 = P[:a_0]
id_0 = P[:id_0]
ys_0 = P[:ys_0]
tm_0 = P[:tm_0]
va_0 = P[:va_0]
md_0 = P[:md_0]
fd_0 = P[:fd_0]
m_0 = P[:m_0]
ty_0 = P[:ty_0]
ms_0 = P[:ms_0]
bopdef_0 = P[:bopdef_0]
x_0 = P[:x_0]
ta_0 = P[:ta_0]
#s_0 = P[:s_0]
fs_0 = P[:fs_0]
y_0 = P[:y_0];


ta = deepcopy(ta_0) *0
ty = deepcopy(ty_0)
tm = deepcopy(tm_0) *0;

1;

In [4]:
yr = Symbol(2017)

WiNnat = MPSGEModel()

y_ = [j for j∈J if sum(ys_0[yr,j,i] for i∈I) !=0]
a_ = [i_ for i_∈I if a_0[yr,i_]!=0]

@sectors(WiNnat,begin
    Y, (index = [y_],)
    A, (index = [a_],)
    MS, (index = [M],)
end)

@commodities(WiNnat,begin
    PA, (index = [a_],)
    PY, (index = [y_],)
    PVA, (index = [VA],)
    PM, (index = [M],)
    PFX
end)

@consumer(WiNnat, RA)


for j∈y_
    add_production!(WiNnat, Y[j],
        ScalarNest(:t; elasticity = 0, children = [
            ScalarOutput(PY[i],ys_0[yr,j,i]; taxes = [Tax(RA,ty[yr,j])]) for i∈I if ys_0[yr,j,i]>0
        ]),
        ScalarNest(:s; elasticity = 0, children = [
            [ScalarInput(PA[i], id_0[yr,i,j]) for i∈a_ if id_0[yr,i,j]>0];
            [ScalarNest(:va; elasticity =  1, children = 
                [ScalarInput(PVA[va], va_0[yr,va,j]) for va∈VA if va_0[yr,va,j]>0])
            ];
        ])
    )
end


for m∈M
    add_production!(WiNnat, MS[m],
        ScalarNest(:t; elasticity = 0, children = [
            ScalarOutput(PM[m], sum(ms_0[yr,i,m] for i∈I))
        ]),
        ScalarNest(:s; elasticity = 0, children = [
            ScalarInput(PY[i], ms_0[yr,i,m]) for i∈I if ms_0[yr,i,m]>0
        ])
    )
end


for i∈a_
    add_production!(WiNnat, A[i],
        ScalarNest(:t; elasticity=2, children = [
            [ScalarOutput(PA[i], a_0[yr,i], taxes=[Tax(RA,ta[yr,i])],reference_price=1-ta_0[yr,i])];
            [ScalarOutput(PFX, x_0[yr,i])]
        ]),
        ScalarNest(:s; elasticity=0, children = [
            [ScalarInput(PM[m], md_0[yr,m,i]) for m∈M if md_0[yr,m,i]>0];
            ScalarNest(:dm; elasticity = 2, children = [
                ifelse(y_0[yr,i]>0, [ScalarInput(PY[i], y_0[yr,i])],[]);
                ifelse(m_0[yr,i]>0, [ScalarInput(PFX, m_0[yr,i], taxes = [Tax(RA,tm[yr,i])],reference_price=1+tm_0[yr,i])],[]);
            ])
        ])
    )
end


add_demand!(WiNnat,
    RA,
    [ScalarDem(PA[i], fd_0[yr,i,:pce]) for i∈a_],
    [
        [ScalarEndowment(PY[i], fs_0[yr,i]) for i∈I];
        [ScalarEndowment(PFX, bopdef_0[yr])];
        [ScalarEndowment(PA[i], -sum(fd_0[yr,i,xfd] for xfd∈FD if xfd!=:pce)) for i∈a_];
        [ScalarEndowment(PVA[va], sum(va_0[yr,va,j] for j∈J)) for va∈VA]
    ]
);

In [5]:
jm = build!(WiNnat)

A JuMP Model
Feasibility problem with:
Variables: 286
`Vector{NonlinearExpr}`-in-`MathOptInterface.Complements`: 279 constraints
`Vector{AffExpr}`-in-`MathOptInterface.Complements`: 7 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 284 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Path 5.0.03
Names registered in the model: A, MS, PA, PFX, PM, PVA, PY, RA, Y, income_balance, market_clearance, zero_profit

In [6]:
fix(get_variable(RA),12453.896315446877; force = true)

solve!(WiNnat; :cumulative_iteration_limit => 10_000)

df = generate_report(WiNnat);

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_A0A5.tmp
 > cumulative_iteration_limit 10000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Preprocessed size   : 285

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             3.7941e+02             0.0e+00 (f[  154])
    1     1     0   285 1.8806e+01  1.0e+00    0.0e+00 (f[  286])
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 1.8806e+01           I 0.0e+00 1.8e+01 (f[  286])
    1     1     3     3 9.8401e-02  1.0e+00 SO 0.0e+00 5.7e-02 (f[  154])
    2     1     4     4 2.7437e-06  1.0e+00 SO 0.0e+00 1.6e-06 (f[  154])
    3     1     5     5 6.9515e-11  1.0e+00 SO 0.0e+00 5.6e-11 (f[  283])

Major Iterations. . . . 3
Minor Iterations. . . . 3
Restarts. . . . . . . . 0
Crash Iterat

In [7]:
df

Row,var,value,margin
,GenericV…,Float64,Float64
1,A[ppd],1.01599,-5.68434e-14
2,A[res],1.03671,-3.41061e-13
3,A[com],1.00095,-3.97904e-13
4,A[amb],0.970346,-4.54747e-13
5,A[fbp],1.04186,2.27374e-13
6,A[rec],1.02538,-2.84217e-14
7,A[con],0.998505,0.0
8,A[agr],1.0236,-5.68434e-14
9,A[eec],1.00721,0.0
